In [1]:
import pandas as pd
import numpy as np

## Reimbursements

This notebook crosses the **CEIS** dataset which gather companies with some type of problem with Federal spheres and reimbursements related to **CEAP**.

In [2]:
reimbursements = pd.read_csv('../data/2016-08-08-current-year.xz',
                                              dtype={'cnpj_cpf': np.str,
                                                     'document_id': np.str,
                                                     'reimbursement_value': np.float},
                                              parse_dates = ['issue_date'], 
                                              low_memory=False)
reimbursements.shape

(185459, 29)

In [3]:
reimbursements.iloc[1]

document_id                                                             5970849
congressperson_name                                           ABEL MESQUITA JR.
congressperson_id                                                        178957
congressperson_document                                                       1
term                                                                       2015
state                                                                        RR
party                                                                       DEM
term_id                                                                      55
subquota_number                                                               1
subquota_description          Maintenance of office supporting parliamentary...
subquota_group_id                                                             0
subquota_group_description                                                  NaN
supplier                                

---
## Inident and Suspended Companies - CEIS

This dataset gather companies and individuals who have suffered sanctions by the organs and entities of the public administration of the various federative spheres.

Origin of the information: Controladoria-Geral da União - CGU (Comptroller General of the Union)

In [4]:
inident_and_suspended_companies = pd.read_csv('../data/2016-12-21-inident-and-suspended-companies.xz',
                                              dtype={'sanctioned_cnpj_cpf': np.str,
                                                     'process_number': np.str},
                                              parse_dates = ['sanction_start_date',
                                                             'sanction_end_date', 
                                                             'data_source_date',
                                                             'published_date'], 
                                              low_memory=False)
inident_and_suspended_companies.fillna('', inplace=True)
inident_and_suspended_companies['sanction_start_date'] = pd.to_datetime(
                            inident_and_suspended_companies['sanction_start_date'], 
                            format='%Y-%m-%d')
inident_and_suspended_companies['sanction_end_date'] = pd.to_datetime(
                            inident_and_suspended_companies['sanction_end_date'],
                            format='%Y-%m-%d')
inident_and_suspended_companies.shape

(13084, 17)

In [5]:
inident_and_suspended_companies.iloc[0]

entity_type                                                                            Física
sanctioned_cnpj_cpf                                                                1540142817
name_given_by_sanctioning_body                                                    01540142817
company_name_receita_database                                               FAIAD HABIB ZAKIR
trading_name_receita_database                                                                
process_number                                                           00001655520138260240
sanction_type                                                  Proibição - Lei de Improbidade
sanction_start_date                                                       2014-08-09 00:00:00
sanction_end_date                                                         2019-08-09 00:00:00
sanctioning_body                            Tribunal de Justiça do Estado de São Paulo / 1...
state_of_sanctioning_body                                   

#### Querying the reimbursements from companies suspended

Below we have a list of reibursements from dates where the suplier is listed in CEIS dataset.

In [6]:
reimbursements_from_inident_companies = reimbursements.merge(
                                        inident_and_suspended_companies, 
                                        left_on='cnpj_cpf', 
                                        right_on='sanctioned_cnpj_cpf')
reimbursements_from_inident_companies.shape

(44, 46)

Now from the selected reimbursements, we get only the ones made between time the saction is valid.

In [7]:
suspect_reimbursements = reimbursements_from_inident_companies.query(
                                           'sanction_start_date <= issue_date and \
                                           issue_date <= sanction_end_date')
suspect_reimbursements.shape

(16, 46)

In [8]:
suspect_reimbursements.filter([ 'document_id',
                                'congressperson_name',
                                'congressperson_id',
                                'congressperson_document',
                                'term',
                                'subquota_description',
                                'subquota_group_description',
                                'supplier',
                                'cnpj_cpf',
                                'document_number',
                                'document_type',
                                'issue_date',
                                'document_value',
                                'name_given_by_sanctioning_body',
                                'company_name_receita_database', 
                                'trading_name_receita_database',
                                'process_number', 
                                'sanction_type', 
                                'sanction_start_date',
                                'sanction_end_date', 
                                'sanctioning_body'])

,document_id,congressperson_name,congressperson_id,congressperson_document,term,subquota_description,subquota_group_description,supplier,cnpj_cpf,document_number,...,issue_date,document_value,name_given_by_sanctioning_body,company_name_receita_database,trading_name_receita_database,process_number,sanction_type,sanction_start_date,sanction_end_date,sanctioning_body
0,5914601,ABEL MESQUITA JR.,178957.0,1.0,2015,"Consultancy, research and technical work",NaN,EFZ COMERCIO E REPRESENTAÇÃO LTDA,16923719000134,2061,...,2016-02-17,20000.00,EFZ COMÉRCIO E REPRESENTAÇÕES LTDA,EFZ COMERCIO E REPRESENTACAO EIRELI - ME,A ! REPRESENTACOES,(não informado),Suspensão - Lei de Licitações,2015-11-30,2017-11-29,FUNDAÇÃO UNIVERSIDADE DE BRASÍLIA
1,5931084,REMÍDIO MONAI,178960.0,7.0,2015,Publicity of parliamentary activity,NaN,EFZ COMERCIO E REPRESENTAÇÃO LTDA,16923719000134,2076,...,2016-03-03,15000.00,EFZ COMÉRCIO E REPRESENTAÇÕES LTDA,EFZ COMERCIO E REPRESENTACAO EIRELI - ME,A ! REPRESENTACOES,(não informado),Suspensão - Lei de Licitações,2015-11-30,2017-11-29,FUNDAÇÃO UNIVERSIDADE DE BRASÍLIA
3,5982793,ADEMIR CAMILO,133374.0,520.0,2015,Congressperson meal,NaN,Reis & Acquaviva Comércio Ltda.,14954288000148,224070,...,2016-03-12,28.00,POUSO ALEGRE FOODS LTDA. ME,POUSO ALEGRE FOODS LTDA - ME,SUBWAY POUSO ALEGRE,10831.722122/2014-35,Suspensão - Lei de Licitações,2015-12-18,2017-12-17,Receita Federal do Brasil
4,5982793,ADEMIR CAMILO,133374.0,520.0,2015,Congressperson meal,NaN,Reis & Acquaviva Comércio Ltda.,14954288000148,224070,...,2016-03-12,28.00,POUSO ALEGRE FOODS LTDA. ME,POUSO ALEGRE FOODS LTDA - ME,SUBWAY POUSO ALEGRE,10814.729285/2014-57,Suspensão - Lei de Licitações,2015-01-23,2017-01-22,Receita Federal do Brasil
5,6045571,ANGELA ALBINO,192120.0,559.0,2015,"Terrestrial, maritime and fluvial tickets",NaN,REUNIDAS TRANSPORTES COLETIVOS,83054395000132,094743,...,2016-06-27,45.75,Reunidas S.A. Transportes de Coletivos,REUNIDAS S A TRANSPORTES COLETIVOS,,50500.044460/2009-23,Inidoneidade - Lei da ANTT e ANTAQ,2014-10-15,2017-10-14,AGÊNCIA NACIONAL DE TRANSPORTES TERRESTRES
6,5972071,CELSO MALDANER,141405.0,472.0,2015,Watercraft renting or charter,NaN,REUNIDAS TRANSPORTES COLETIVOS,83054395000132,777393,...,2016-04-09,51.93,Reunidas S.A. Transportes de Coletivos,REUNIDAS S A TRANSPORTES COLETIVOS,,50500.044460/2009-23,Inidoneidade - Lei da ANTT e ANTAQ,2014-10-15,2017-10-14,AGÊNCIA NACIONAL DE TRANSPORTES TERRESTRES
7,5972059,CELSO MALDANER,141405.0,472.0,2015,Watercraft renting or charter,NaN,REUNIDAS TRANSPORTES COLETIVOS,83054395000132,777394,...,2016-04-10,111.35,Reunidas S.A. Transportes de Coletivos,REUNIDAS S A TRANSPORTES COLETIVOS,,50500.044460/2009-23,Inidoneidade - Lei da ANTT e ANTAQ,2014-10-15,2017-10-14,AGÊNCIA NACIONAL DE TRANSPORTES TERRESTRES
31,5901395,MARCOS MONTES,141500.0,257.0,2015,Maintenance of office supporting parliamentary...,NaN,JEREMAIS RAIMUNDO VENANCIO,64183858615,01/2016,...,2016-02-01,1200.00,JEREMIAS RAIMUNDO VENâNCIO,JEREMIAS RAIMUNDO VENANCIO,,0040090953387,Proibição - Lei de Improbidade,2015-12-16,2018-12-16,Tribunal de Justiça do Estado de Minas Gerais ...
32,5986940,MARCOS MONTES,141500.0,257.0,2015,Maintenance of office supporting parliamentary...,NaN,JEREMAIS RAIMUNDO VENANCIO,64183858615,sn,...,2016-05-02,1200.00,JEREMIAS RAIMUNDO VENâNCIO,JEREMIAS RAIMUNDO VENANCIO,,0040090953387,Proibição - Lei de Improbidade,2015-12-16,2018-12-16,Tribunal de Justiça do Estado de Minas Gerais ...
33,6010353,MARCOS MONTES,141500.0,257.0,2015,Maintenance of office supporting parliamentary...,NaN,JEREMAIS RAIMUNDO VENANCIO,64183858615,sn,...,2016-05-31,1200.00,JEREMIAS RAIMUNDO VENâNCIO,JEREMIAS RAIMUNDO VENANCIO,,0040090953387,Proibição - Lei de Improbidade,2015-12-16,2018-12-16,Tribunal de Justiça do Estado de Minas Gerais ...


The study above doesnt prove anything ilegal, but at least the congressperson should have research about the company before use its services.